In [ ]:
#PRIMER SCRIPT: REALIZA UNA REGRESIÓN LINEAL SIMPLE
# Este script realiza una regresión simple propuesta en clase.
# Usa como variables los "mágicos", que son ciertos productos seleccionados para el análisis.

import pandas as pd
import numpy as np
import gc
from sklearn.linear_model import LinearRegression

# Carga del dataset de ventas
df = pd.read_csv("sell-in.txt", sep="\t")

# Se agrupa por periodo y producto, sumando las toneladas vendidas (tn)
df = df.groupby(by=["periodo", "product_id"]).agg({"tn": "sum"}).reset_index()

# Se convierte la columna 'periodo' a formato datetime
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")

# Reestructuración del DataFrame: cada fila es un mes, cada columna un product_id, valores = tn
df_pivot = df.pivot(index="periodo", columns="product_id", values="tn").reset_index()

# 5. Lista de productos mágicos seleccionados para el análisis (más la columna 'periodo')
magicos = [ 
    "periodo", 20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
    20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046, 20049,
    20051, 20052, 20053, 20055, 20008, 20001, 20017, 20086, 20180,
    20193, 20320, 20532, 20612, 20637, 20807, 20838
]

# Se construye el conjunto de entrenamiento (X_train) con datos del año 2018
X_train = df_pivot[magicos].query("periodo >= '2018-01-01' & periodo <= '2018-12-31'")
X_train = X_train.T.iloc[1:]  # Transpone y elimina la fila 'periodo'
X_train.columns = [f"t-{11-k}" for k in range(12)]  # Etiquetas de los 12 meses hacia atrás

# Se construye el conjunto X_kgl con los datos de 2019, para predicción
X_kgl = df_pivot.query("periodo >= '2019-01-01' & periodo <= '2019-12-31'")
X_kgl = X_kgl.T.iloc[1:]  # Transpone y elimina la fila 'periodo'
X_kgl.columns = [f"t-{11-k}" for k in range(12)]

# Se calcula el promedio por producto (fila) como baseline, por si faltan datos
promedio = X_kgl.mean(axis=1).fillna(0)

# Se crea el vector objetivo (y) usando las toneladas de febrero 2019
y = df_pivot[magicos].query("periodo == '2019-02-01'").T.iloc[1:]
y.columns = ["target"]

# Se eliminan productos que no tengan 12 meses de datos (tienen NaN)
prod_menos12 = X_kgl.index[X_kgl.isna().sum(axis=1) > 0]
X_kgl = X_kgl[~X_kgl.index.isin(prod_menos12)]  # Solo productos con info completa
promedio_menos12 = promedio[prod_menos12]  # Guarda promedio para los que tienen datos faltantes

# Se carga la lista de productos a predecir desde el archivo externo
productos_ok = pd.read_csv(
    "https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", 
    sep="\t"
)

# Se entrena un modelo de regresión lineal usando X_train e y
reg_model = LinearRegression()
reg_model.fit(X_train, y)

# Se realiza la predicción para los productos que tienen datos completos
pred = pd.DataFrame({
    "product_id": X_kgl.index,
    "tn": reg_model.predict(X_kgl).flatten()
})

# Para los productos faltantes, se completa con el promedio del año
nuevas_filas = []
for prod in productos_ok["product_id"]:
    if prod not in pred["product_id"].values:
        nuevas_filas.append({
            "product_id": prod, 
            "tn": promedio[prod]
        })

# Se unen las predicciones del modelo con los productos completados por promedio
pred = pd.concat([pred, pd.DataFrame(nuevas_filas)], ignore_index=True)

# Se aseguran solo los productos requeridos en la lista final
pred = pred[pred["product_id"].isin(productos_ok["product_id"])]

# Exportación de las predicciones a un archivo CSV 
pred.to_csv("prediccion_reg_lineal.csv", index=False, sep=",")


In [ ]:
#SEGUNDO SCRIPT: AUTOGLUON
#El mismo script que fue compartido en zulip por Fernando Raco, tal cual lo compartió él (solo unificado) 
# y que estuvimos viendo el clase, de probada funcionalidad. 

# 📦 1. Importar librerías
import pandas as pd
# 💬 Instalar AutoGluon si es necesario
%pip install autogluon.timeseries

from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

# 📄 2. Cargar datasets
df_sellin = pd.read_csv("sell-in.txt", sep="\t")
df_productos = pd.read_csv("tb_productos.txt", sep="\t")

# 📄 Leer lista de productos a predecir
with open("product_id_apredecir201912.TXT", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]
    
    # 🧹 3. Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

# Filtrar hasta dic 2019 y productos requeridos
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-01') &
    (df_sellin['product_id'].isin(product_ids))
]

# Agregar tn por periodo, cliente y producto
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()

# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

# ⏰ 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

# ⚙️ 5. Definir y entrenar predictor
predictor = TimeSeriesPredictor(
    prediction_length=2,
    target='tn',
    freq='MS'  # Frecuencia mensual (Month Start), 
)

predictor.fit(ts_data, num_val_windows=2, time_limit=60*60)

# 🔮 6. Generar predicción
forecast = predictor.predict(ts_data)

# Extraer predicción media y filtrar febrero 2020
forecast_mean = forecast['mean'].reset_index()
print(forecast_mean.columns)

# Tomar solo item_id y la predicción 'mean'
resultado = forecast['mean'].reset_index()[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']

# Filtrar solo febrero 2020
resultado = forecast['mean'].reset_index()
resultado = resultado[resultado['timestamp'] == '2020-02-01']

# Renombrar columnas
resultado = resultado[['item_id', 'mean']]
resultado.columns = ['product_id', 'tn']


# 💾 7. Guardar archivo
resultado.to_csv("predicciones_febrero2020_fecha_01_07.csv", index=False)
resultado.head()

In [ ]:
#TERCER SCRIPT: COMPARATIVA DE MODELOS
# Este script ejecuta una comparativa de modelos, con validación ampliada a septiembre, octubre y noviembre. 0.260 en el public
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
import lightgbm as lgb
import xgboost as xgb
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

warnings.filterwarnings("ignore")

# 1. Cargar dataset
df = pd.read_csv("sell-in.txt", sep="\t")
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m')
df = df.groupby(['product_id', 'periodo'])['tn'].sum().reset_index()

# 2. Cargar listado fijo de productos
with open("product_id_apredecir201912.TXT", "r") as f:
    productos = [int(line.strip()) for line in f if line.strip().isdigit()]

# Se inicializa la salida
resultados = []
log = []
maes_resumen = []

# Se crea carpeta par autogluon
os.makedirs("autogluon_temp_ts", exist_ok=True)

productos_predichos = set()

# Se filtran los datos del df, ordenándolos por fecha y extrayendo el mes como única feature.
# Se dividen los datos en un conjunto de entrenamiento (todo antes de septiembre de 2019) y otro de validación (solo septiembre, octubre y noviembre de 2019)
# Se preparan los datasets X_train y X_val usando únicamente el mes como variable explicativa, y y_train y y_val con las toneladas vendidas. 
for prod in tqdm(productos, desc="Procesando productos"):
    datos = df[df['product_id'] == prod].sort_values('periodo').copy()
    datos['mes'] = datos['periodo'].dt.month

    train = datos[datos['periodo'] < '2019-09-01'].copy()
    val = datos[datos['periodo'].isin([
        pd.Timestamp('2019-09-01'),
        pd.Timestamp('2019-10-01'),
        pd.Timestamp('2019-11-01')
    ])].copy()

    if len(train) < 12 or val.empty:
        continue

    X_train = train[['mes']]
    y_train = train['tn']
    X_val = val[['mes']]
    y_val = val['tn']

    maes = {}
    preds = {}

    # Se aplica el primer modelo de Regresión lineal
    try:
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        y_pred = lr.predict(X_val)
        maes['regresion'] = mean_absolute_error(y_val, y_pred)
        preds['regresion'] = lr.predict([[2]])[0]
    except:
        maes['regresion'] = np.inf

    # Se aplica el segundo modelo: ARIMA
    try:
        serie = train.set_index('periodo')['tn']
        modelo_arima = ARIMA(serie, order=(1, 1, 1)).fit()
        y_pred = modelo_arima.forecast(steps=3)
        maes['arima'] = mean_absolute_error(y_val.values, y_pred.values)
        feb_pred = modelo_arima.forecast(steps=5)[-1]
        preds['arima'] = feb_pred
    except:
        maes['arima'] = np.inf

    # Se aplica el tercer modelo, LightGBM. Los hiperparámetros son ajustados en función de una optimización previa con Optuna.
    try:
        lgb_model = lgb.LGBMRegressor(
            n_estimators=834,
            learning_rate=0.06449926163783713,
            max_depth=13,
            num_leaves=197,
            min_data_in_leaf=208,
            min_child_weight=3.7932779938198546,
            subsample=0.7032151245633396,
            subsample_freq=7,
            colsample_bytree=0.9893937066314805,
            colsample_bynode=0.8148358693555268,
            reg_alpha=4.962755134948597,
            reg_lambda=3.8191748367071927,
            max_bin=512,
            min_split_gain=0.006311109685921704,
            cat_smooth=49.82693114488869,
            random_state=42,
            boosting_type='dart',
            verbosity=-1,
            linear_tree=True
        )
        lgb_model.fit(X_train, y_train)
        y_pred = lgb_model.predict(X_val)
        maes['lgbm'] = mean_absolute_error(y_val, y_pred)
        preds['lgbm'] = lgb_model.predict([[2]])[0]
    except:
        maes['lgbm'] = np.inf

    # Se aplica el cuarto modelo: XGBoost
    try:
        xgb_model = xgb.XGBRegressor(verbosity=0)
        xgb_model.fit(X_train, y_train)
        y_pred = xgb_model.predict(X_val)
        maes['xgboost'] = mean_absolute_error(y_val, y_pred)
        preds['xgboost'] = xgb_model.predict([[2]])[0]
    except:
        maes['xgboost'] = np.inf

    # Se aplica el quinto modelo, AutoGluon, tomando como base el script que se había probado funcional en clase.
    try:
        df_serie = train[['periodo', 'tn']].copy()
        df_serie['item_id'] = str(prod)
        df_serie = df_serie.rename(columns={'periodo': 'timestamp'})
        df_serie = df_serie[['item_id', 'timestamp', 'tn']]

        ts_data = TimeSeriesDataFrame.from_data_frame(
            df_serie, id_column='item_id', timestamp_column='timestamp'
        ).fill_missing_values()

        predictor = TimeSeriesPredictor(
            prediction_length=5,
            target='tn',
            freq='MS',
            eval_metric='MASE',
            path=f"autogluon_temp_ts/{prod}",
            verbosity=0
        )

        predictor.fit(
            ts_data,
            num_val_windows=2,
            time_limit=60,
            enable_ensemble=False,
            hyperparameters={"ETS": {}, "AutoARIMA": {}, "Naive": {}}
        )

        forecast = predictor.predict(ts_data)
        val_preds = [forecast.loc[(str(prod), pd.Timestamp(d)), 'mean'] for d in ['2019-09-01', '2019-10-01', '2019-11-01']]
        maes['autogluon'] = mean_absolute_error(y_val, val_preds)
        preds['autogluon'] = forecast.loc[(str(prod), pd.Timestamp("2020-02-01")), 'mean']
    except:
        maes['autogluon'] = np.inf

   # Este bloque selecciona el mejor modelo, basándose en el error MAE.  
    mejor_modelo = min(maes, key=maes.get)
    pred_final = preds[mejor_modelo]
    resultados.append({'product_id': prod, 'tn_predicho': pred_final})
    productos_predichos.add(prod)
    log.append(f"Producto {prod}: mejor modelo = {mejor_modelo}, MAE sep-nov = {maes[mejor_modelo]:.4f}")

    mae_row = {'product_id': prod}
    for modelo in ['regresion', 'arima', 'lgbm', 'xgboost', 'autogluon']:
        mae_row[f'mae_{modelo}'] = maes.get(modelo, np.nan)
    maes_resumen.append(mae_row)

# Para los productos que no pudieron ser modelados en el paso anterior (por falta de datos o por error)
# se aplica una estrategia de fallback para asegurar que todos los productos tengan una predicción.
# Se calcula el promedio de las toneladas vendidas en los últimos 12 meses antes de septiembre
productos_faltantes = set(productos) - productos_predichos
for prod in productos_faltantes:
    datos = df[df['product_id'] == prod].sort_values('periodo').copy()
    ultimos_12 = datos[datos['periodo'] < '2020-01-01'].tail(12)
    pred_fallback = ultimos_12['tn'].mean() if not ultimos_12.empty else 0
    resultados.append({'product_id': prod, 'tn_predicho': pred_fallback})
    log.append(f"Producto {prod}: fallback promedio últimos 12 meses = {pred_fallback:.2f}")

# Se guardan los resultados en archivos csv 
pd.DataFrame(resultados).sort_values("product_id").to_csv("predicciones_febrero2020_porproducto3.csv", index=False)
maes_df = pd.DataFrame(maes_resumen).sort_values("product_id")
maes_df.to_csv("maes_por_modelo.csv", index=False)
with open("log_modelos3.txt", "w") as f:
    for linea in log:
        f.write(linea + "\n")




In [ ]:
#CUARTO SCRIPT: ENSEMBLE DE RESULTADOS
# Este script combina las predicciones de varios modelos para obtener una predicción final promediada. Se utilizan para ello
# los acrchivos generados por los scripts anteriores. Y se combinan con un cuarto, que es el resultado del modelo elegido por Pablo Cablinski.
# de ese modo se presenta un csv que es el elegido por el grupo para la entrega final del laboratorio. En el public performa 0.242

# Cargar archivos
df1 = pd.read_csv("predicciones_febrero2020_porproducto3.csv")
df2 = pd.read_csv("prediccion_reg_lineal.csv")
df3 = pd.read_csv("lgbm_predictions_median.csv")
df4 = pd.read_csv("predicciones_febrero2020_autogluonFernando.csv")


# Renombrar columnas para consistencia
df1.columns = ["product_id", "tn_predicho"]
df2.columns = ["product_id", "tn_predicho"]
df3.columns = ["product_id", "tn_predicho"]
df4.columns = ["product_id", "tn_predicho"]


# Unir por product_id
df_merge = df1.merge(df2, on="product_id", suffixes=("_1", "_2"))
df_merge = df_merge.merge(df3, on="product_id")
df_merge.rename(columns={"tn_predicho": "tn_predicho_3"}, inplace=True)
df_merge = df_merge.merge(df4, on="product_id")
df_merge.rename(columns={"tn_predicho": "tn_predicho_4"}, inplace=True)



# Calcular promedio
df_merge["tn_predicho"] = df_merge[
    ["tn_predicho_1", "tn_predicho_2", "tn_predicho_3", "tn_predicho_4"]
].mean(axis=1)

# Mergear las predicciones del modelo
df_merge["tn_predicho"] 

# Seleccionar columnas finales
df_final = df_merge[["product_id", "tn_predicho"]]

# Guardar archivo final
df_final.to_csv("predicciones_promediadas_pw.csv", index=False)
print("Archivo guardado como predicciones_promediadas_pw.csv")
